In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "../Resources/purchase_data.csv"


In [2]:
# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
# rename column
purchase_data = purchase_data.rename(columns={"SN":"User"})
purchase_data.head()

,Purchase ID,User,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
# Check to see if there are any rows with missing data
purchase_data.count()

##   DATA IS CLEAN   ##

Purchase ID    780
User           780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [4]:
# Player Count
count = purchase_data["User"].unique()
print ("Total Number of Players are:", len(count))

Total Number of Players are: 576


In [5]:
## Purchasing Analysis (Total)
#count1 = purchase_data["Item ID"].unique()
#average = float(purchase_data["Price"].mean())
# count2 = purchase_data["Purchase ID"].unique()
#revenue = float(purchase_data["Price"].sum())

In [6]:
# Purchasing Analysis (Total)
# creating df Purchasing Analysis (Total) results
results_0 = pd.DataFrame({
    "Number of Unique Items": [len(purchase_data["Item ID"].unique())],
    "Average Price": [float(purchase_data["Price"].mean())],
    "Number of Purchases": [len(purchase_data["Purchase ID"].unique())],
    "Total Revenue": [float(purchase_data["Price"].sum())]
})
results_0["Average Price"] = results_0["Average Price"].map("${:,.2f}".format)
results_0["Total Revenue"] = results_0["Total Revenue"].map("${:,.2f}".format)
results_0

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [7]:
# # Gender Demographics
user_gender_df = purchase_data[["Gender","User"]].drop_duplicates(subset = "User")
gender_count_df = pd.DataFrame(user_gender_df["Gender"].value_counts(0))
gender_perc_df = pd.DataFrame(user_gender_df["Gender"].value_counts(1))

In [8]:
# creating df # Gender Demographics results
results_1 = gender_count_df.merge(gender_perc_df, left_index = True, right_index = True)
results_1.columns = ["Total Count", "Percentage of Players"]
results_1['Percentage of Players'] = results_1['Percentage of Players'].map("{:,.2%}".format)
results_1

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [9]:
# Purchasing Analysis (Gender)
price_gender_df = purchase_data.groupby("Gender")

purchase_count = round(price_gender_df["Purchase ID"].count(),0)
average_count = round(price_gender_df["Price"].mean(),2)
total_count = round(price_gender_df["Price"].sum(),2)
average_person = (total_count / gender_count_df["Gender"]).map("${:.2f}".format)

In [10]:
# creating df Purchasing Analysis (Gender) results
purchase_count.name = "Purchase Count"
average_count.name = "Average Purchase Price"
total_count.name = "Total Purchase Value"
average_person.name = "Avg Total Purchase total Person"

results_2 = pd.concat([purchase_count, average_count,
    total_count, average_person,],axis=1)

results_2["Average Purchase Price"]=results_2["Average Purchase Price"].map("${:.2f}".format)
results_2["Total Purchase Value"]=results_2["Total Purchase Value"].map("${:.2f}".format)
results_2

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase total Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [11]:
# Age Demographics
bins = [0, 9, 14, 19, 24, 29, 34, 39, 120]
group_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

user_list = purchase_data.loc[:,["User","Age"]]
user_list_clean = user_list.drop_duplicates() # remove all duplicates to keep one record per user

#using .cut
user_list_clean= pd.cut(user_list_clean["Age"],bins,labels = group_names)

In [12]:
# creating df Age Demographics results
results_3 = pd.DataFrame({"Total Count": 
                          user_list_clean.value_counts(),
                          "Percentage of Players": 
                          (user_list_clean.value_counts()/len(count))*100})
results_3 = results_3.sort_index()
results_3["Percentage of Players"] = results_3["Percentage of Players"].map("{:,.2f}%".format)
results_3

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [13]:
#Purchasing Analysis (Age)
#using .cut
purchase_data["column"] = pd.cut(purchase_data["Age"],bins,labels = group_names)

In [14]:
# creating df Purchasing Analysis (Age) results
results_4 = pd.DataFrame({"Purchase Count": 
                          purchase_data.groupby(["column"]).count()["Price"],
                          "Average Purchase Price": 
                          purchase_data.groupby(["column"]).mean()["Price"],
                          "Total Purchase Value": 
                          purchase_data.groupby(["column"]).sum()["Price"],
                          "Average Total Purchase per Person": 
                          purchase_data.groupby(["column"]).sum()["Price"]/user_list_clean.value_counts()})
                                  
results_4["Average Purchase Price"] = results_4["Average Purchase Price"].map("${:,.2f}".format)
results_4["Total Purchase Value"] = results_4["Total Purchase Value"].map("${:,.2f}".format)
results_4["Average Total Purchase per Person"] = results_4["Average Total Purchase per Person"].map("${:,.2f}".format)
results_4.index.name="Age Ranges"                                  
results_4   

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [43]:
# Top Spenders
Purchase_user_count = purchase_data.groupby(["User"]).count()["Price"]
Purchase_user_aveg = purchase_data.groupby(["User"]).mean()["Price"]
Purchase_user_total = purchase_data.groupby(["User"]).sum()["Price"]

In [44]:
# creating df Top Spenders results
results_5 =pd.DataFrame({"Purchase Count":
                         purchase_data.groupby(["User"]).count()["Price"],
                         "Average Purchase Price":
                         purchase_data.groupby(["User"]).mean()["Price"],
                         "Total Purchase Value":
                         purchase_data.groupby(["User"]).sum()["Price"]})
results_5_1 =results_5.sort_values('Total Purchase Value', ascending=False).reset_index()
results_5_1["Average Purchase Price"] = results_5_1["Average Purchase Price"].map("${:,.2f}".format)
results_5_1["Total Purchase Value"] = results_5_1["Total Purchase Value"].map("${:,.2f}".format)
results_5_1.head()

,User,Purchase Count,Average Purchase Price,Total Purchase Value
0,Lisosia93,5,$3.79,$18.96
1,Idastidru52,4,$3.86,$15.45
2,Chamjask73,3,$4.61,$13.83
3,Iral74,4,$3.41,$13.62
4,Iskadarya95,3,$4.37,$13.10


In [16]:
# Most Popular Items

In [17]:
# Most Profitable Items